# Building a simple Coup game

Coup is a popular board game which has a fast pace. The description of the game can be found at this website: https://boardgamegeek.com/boardgame/131357/coup. Basically there are five different cards and each turn a player can make a single decision. I divided the coding procedure into 7 parts:


1. Define player
2. Define actions
3. Simulate the game for human players
4. Develop AI
5. Come up with a strategy for AI
6. Upgrade to a multi-player set
7. Define a lying game


In [1]:
# Import packages
import sys                        # for executing the program
from random import shuffle        # for deck shuffling

# Part one: Define player

I first create a game with only three players. Player will be a class distinguished by their names which has instance coin (initially 2) and hands. Hands should be a dictionary with a maximum length two. If the player has no cards he will die.

In [17]:
class Player:

    
    # variable is assigned by given a player name, initialize 2 coins and an empty hand
    def __init__(self,name):
        self.name = name
        self.coins = 2
        self.hands = []
        
    # function to show players their current card
    def show_hand(self):
        print('Your hands are:')
        for hand in self.hands:
            print(hand)
        print('\n')    
    

# Part two: Define actions

Before define the actual game actions, we need to define some basic actions firsts. 

We first define a function to check whether the game stops, this function will be triggered everytime player show down their card.

In [27]:
def endgame_check():
    global players
    if len(players) == 1:
        for player in players:
            winner = player.keys()
        print('The winner of the game is ' + winner + '. Congratulations!')
        sys.exit()
    else: pass

There should also be a function for a player choosing one card to loose(which also means you lose one life), after this action, we always check if the player is still alive and if the game ends.

In [6]:
def discard(victim):
    
    global players
    
    card_discard = input('Player '+victim+', please select one card to show down:')
    
    while not card_discard in players[victim].hands:
        card_discard = input('This is not the right place to lie, please re-enter:')
    
    players[victim].hands.remove(card_discard)
    discarded_pool.append((victim,card_discard))
    
    if players[victim].hands == []:
        del players[victim]
        
    endgame_check()
            

Define a function to exchange hand with pool, note that you might get back the same card. After this the player will check his hands.

In [38]:
def exchange(player,card):
    player.hands.remove(card)
    pool.append(card)
    shuffle(pool)
    new_card = pool.pop()
    player.hands.append(new_card)
    shuffle(pool)
    input(player.name+', Press any key to see your hand')
    player.show_hand()

Since for all actions, other players will be asked whether to qustion or not. So we create question function which
will only be called with action methods. Players will be asked in turn whether they want to question. If nobody questions the function does nothing. If somebody questions the funtion will judge whether this quesiton is successful or not. I call the input of this function 'actioner', while who question this 'questioner'.

When questioning or blocking, we need an iterator which takes the seccond next player and loop till the player before him. So we create a function to generate the iterator.

In [37]:
def calling(actioner):
    global players
    players_name = [name for name in players.keys()]*2
    initial = players_name.index(actioner.name)+1
    iterator = []
    for i in range(initial,initial+len(players)-1):
        iterator.append(players[players_name[i]])
    return iterator

In [36]:
def question(actioner,card):
    iterator = calling(actioner)
    for questioner in iterator:
        respond = input('Player '+questioner.name+', do you what to question? [yes/no]')
        if respond == 'yes':
            if card in player.hands:
                discard(questioner)
                exchange(actioner,card)
                continue_action = True
                return continue_action
            else:
                discard(actioner)
                continue_action = False
                return continue_action
    continue_action = True
    return continue_action

We put all possible actions inside a function called action, the input will be the player and name of the action, then different effects will happen according to what the player inputs. To make things simple, we first create an action list for the future check.

In [15]:
action_list = ['income','foreign aid','coup','duke','assassin','duke','ambassador','captain']

In [26]:
def take_action(player,action):
    
    
    
    if action == 'income':
        player.coin += 1
    
    
    
    elif action == 'foreign aid':
        iterator = calling(player)
        for blocker in iterator:
            respond = input('Player '+blocker.name+', do you what to block? [yes/no]')
            if respond == 'yes':
                continue_action = question(blocker,'duke')
                if continue_action:
                    return
                else: 
                    player.coins += 2
                    return
        player.coins += 2
        return
      
        
        
    elif action == 'coup':
        victim = input('please select a player to coup:')        
        while not victim in players:
            victim = input('Player doesn\'t exit, please re-enter:')
        discard(victim)        
        player.coins -= 7
        
        
        
    elif action == 'duke':
        continue_action = question(player,'duke')
        if continue_action:
            player.coins += 3
        
        return
        
        
        
    elif action == 'assassin':
        player.coins -= 3
        victim = input('Who do you want to assassinate:')
        while not victim in players:
            victim = input('Player doesn\'t exit, please re-enter:')
        continue_action = question(player,'assassin')
        if continue_action:
            defend = input('Player '+victim.name+', do you want to play contessa? [yes/no]')
            if defend == 'yes':
                continue_action_2 = question(victim,'contessa')
                
                if continue_action_2:
                return
            
            discard(victim)
            return
        return
    
    
    
    elif action == 'captain':
        target = input('Who do you want to steal:')
        while not target in players:
            target = input('Player doesn\'t exit, please re-enter:')
        continue_action = question(player,'captain')
        if continue_action:
            defend = input('Player '+target.name+', do you want to block stealing? [yes/no]')
            if defend == 'yes':
                iterator = calling(target)
                for questioner in iterator:
                    respond = input('Player '+questioner.name+', do you what to question? [yes/no]')
                    if respond == 'yes':
                        if 'captain' in player.hands:
                            discard(questioner)
                            exchange(target,'captain')
                            return
                        elif 'ambassador' in player.hands:
                            discard(questioner)
                            exchange(target,'ambassador')
                            return
                        else:
                            discard(target)
                            target.coins -= 2
                            player.coins += 2
                            return
                return    
            target.coins -= 2
            player.coins += 2
            return
        return
    
    
    
    
    else:
        continue_action = question(player,'ambassador')
        if continue_action:
            for _ in range(2):
                new_card = pool.pop()
                player.hands.append(new_card)        
            input(player.name+', Press any key to see your hand')
            player.show_hand()
            
            put_back = input('Please select the first card you want to put back:')
            while not put_back in players.hand:
                put_back = input('Please put back a card you own:') 
            player.hands.remove(put_back)
            pool.append(put_back)
            
            put_back = input('Please select the second card you want to put back:')
            while not put_back in players.hand:
                put_back = input('Please put back a card you own:') 
            player.hands.remove(put_back)
            pool.append(put_back)
            
            shuffle(pool)
    
                
            
            

Player b, do you what to question? [yes/no]yes


# Part three: create a game

We should have card pool since the player will exchange cards with the pool. 

We assume there are infinitely many coins in the bank.

We also need a termination of the game. The game has only one survivor so we will end this game if the number of players is one.

# Configuration

In [39]:
# Players login
number_of_players = input('Please enter the number of total players:\n')
number_of_players = int(number_of_players)

# if number_of_players <= 2:
#     print('At least three players required for this game')
#     sys.exit()


players = {}

for i in range(number_of_players):
    name = input('Player {}, please enter your player name:'.format(i+1))
    players[name] = Player(name)

print('\n')

# Create the card pool

pool = ['assassin','duke','contessa','ambassador','captain']*3

shuffle(pool)

# Card dealing

for player in players.values():
    for _ in range(2):
        new_card = pool.pop()
        player.hands.append(new_card)        
    
    input(player.name+', Press any key to see your hand')
    player.show_hand()

# Create discarded cards, this will be a list of (player name, card name) tupple

discarded_pool = []


Please enter the number of total players:
3
Player 1, please enter your player name:a
Player 2, please enter your player name:b
Player 3, please enter your player name:c


a, Press any key to see your hand
Your hands are:
ambassador
contessa


b, Press any key to see your hand
Your hands are:
duke
assassin


c, Press any key to see your hand
Your hands are:
captain
duke




# Game phase

1. check whether player is forced to coup (coins >= 10)
2. select an action
3. check whether game stops (len(players) == 1)

In [7]:
for player in players.values():
    print('Player '+player.name+',it\'s your turn.')
    
    # show the discarded card pool for reference, this is probably the best time to show it.
    print(discarded_pool)
    
    # force coup check
    if player.coins >= 10:
        victim = input('You are forced to coup, please select a player to coup:')
        
        while not victim in players:
            victim = input('Player doesn\'t exit, please re-enter:')

        discard(victim)
        
        player.coins -= 7
        
    else: action = input('Please select an action:')
        
    while not action in action_list:
        action = input('Please re-enter a valid action:')
    take_action(player,action)

Player a,it's your turn.
Player b,it's your turn.
Player c,it's your turn.


In [18]:
a  = Player('a')

# Future improvements

1. The iterator definitely needs a big fix, first it needs to be more consise. Also the iterator should start at the correct position for blocker and contessa 
2. Adding GUI (change input string to button for instance)
3. Use machine learning to improve AI performance
4. Break down the code to different files that only the main file will be executed
5. Change to a design which allows new action/card to be added in easily